# Video face tracking and counting
## with gender and age predict

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Video-face-tracking-and-counting" data-toc-modified-id="Video-face-tracking-and-counting-1">Video face tracking and counting</a></span><ul class="toc-item"><li><span><a href="#with-gender-and-age-predict" data-toc-modified-id="with-gender-and-age-predict-1.1">with gender and age predict</a></span><ul class="toc-item"><li><span><a href="#Import" data-toc-modified-id="Import-1.1.1">Import</a></span></li><li><span><a href="#Загружаем-предобученные-модели-(age_model_UTKFace.h5-и-gen_model_UTKFace.h5)" data-toc-modified-id="Загружаем-предобученные-модели-(age_model_UTKFace.h5-и-gen_model_UTKFace.h5)-1.1.2">Загружаем предобученные модели (age_model_UTKFace.h5 и gen_model_UTKFace.h5)</a></span></li><li><span><a href="#Загружаем-предобученную-модель-детектора-лица" data-toc-modified-id="Загружаем-предобученную-модель-детектора-лица-1.1.3">Загружаем предобученную модель детектора лица</a></span></li><li><span><a href="#Face-MultiTracker" data-toc-modified-id="Face-MultiTracker-1.1.4">Face MultiTracker</a></span></li></ul></li></ul></li></ul></div>

### Import

In [1]:
import os
import sys
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from tqdm import tqdm
import time

from tensorflow.keras.models import load_model
import tensorflow as tf

import cv2
# np.random.seed(42)

cv2.__version__

'4.7.0'

### Загружаем предобученные модели (age_model_UTKFace.h5 и gen_model_UTKFace.h5)

In [2]:
loaded_age_model = tf.keras.models.load_model('age_model_UTKFace.h5')
loaded_gen_model = tf.keras.models.load_model('gen_model_UTKFace.h5')

### Загружаем предобученную модель детектора лица

In [3]:
face_cascade = cv2.CascadeClassifier('cascades/haarcascade_frontalface_default.xml')

### Face MultiTracker 

In [5]:
trackerTypes = ['BOOSTING', 'MIL', 'KCF', 'TLD', 'MEDIANFLOW', 'GOTURN', 'MOSSE', 'CSRT']

def createTrackerByName(trackerType):
    ''' Create tracker'''
    if trackerType == trackerTypes[0]:
        tracker = cv2.legacy.TrackerBoosting_create()
    elif trackerType == trackerTypes[1]:
        tracker = cv2.legacy.TrackerMIL_create()
    elif trackerType == trackerTypes[2]:
        tracker = cv2.legacy.TrackerKCF_create()
    elif trackerType == trackerTypes[3]:
        tracker = cv2.legacy.TrackerTLD_create()
    elif trackerType == trackerTypes[4]:
        tracker = cv2.legacy.TrackerMedianFlow_create()
    elif trackerType == trackerTypes[5]:
        tracker = cv2.legacy.TrackerGOTURN_create()
    elif trackerType == trackerTypes[6]:
        tracker = cv2.TrackerMOSSE_create()
    elif trackerType == trackerTypes[7]:
        tracker = cv2.legacy.TrackerCSRT_create()
    else:
        tracker = None
        print('Incorrect tracker name')
        print('Available trackers are:')
        for t in trackerTypes:
            print(t)
    return tracker

def get_iou(boxA, boxB):
    '''Расчет метрики IoU
    boxA = (x,y,w,h), boxA = (x,y,w,h)'''
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[0]+boxA[2], boxB[0]+boxB[2])
    yB = min(boxA[1]+boxA[3], boxB[1]+boxB[3])
    # compute the area of intersection rectangle
    interArea = max(0, xB - xA) * max(0, yB - yA)
    # compute the area of both the prediction and ground-truth rectangles
    boxAArea = boxA[2] * boxA[3]
    boxBArea = boxB[2] * boxB[3]
    # iou
    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou

def pred_gender_age(x,y,w,h):
    ''' predict age and gender
    x,y,w,h - face_box or tracker_box'''
    # gray face image
    face_img=gray[y:y+w,x:x+w]
    img=cv2.resize(face_img,(64,64))
    img=img/255.0
    pred_age = loaded_age_model.predict(tf.expand_dims(img, axis=0))
    pred_gen = loaded_gen_model.predict(tf.expand_dims(img, axis=0))
    label_gen = np.argmax(pred_gen,axis=1)[0]
    label_age = np.round(pred_age[0][0])
    return label_gen, label_age

# count_dict
count_dict = {}

def count_by_gender(count_dict):
    '''count_dict = {0: ['Male', 32.0]}'''
    count_male = 0
    count_female = 0

    for val in count_dict.values():
        if val[0] == 'Male':
            count_male +=1
        else:
            count_female +=1
    return count_male, count_female
########################################################################

# gender and color dict
gender_dict={0:'Male',1:'Female'}
color_dict={0:(0,0,255),1:(0,255,0)}

# Тип трекера
trackerType = "KCF"

# Словарь лиц
face_dict = {}
face_id = 0
trashold_face_loss = 5
trashold_iou = 0.5

# Start time
num_frames = 0
start = time.time()
fps = 0

cam = cv2.VideoCapture('test1.mp4')
# 'test1.mp4'

# output = cv2.VideoWriter('Test_count_people.mp4', cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), 20, frame_size)


while (cam.isOpened()):
    
    ret, frame = cam.read()
    # выход, если видео недоступно
    if not ret:
        print('Failed to read video')
        break
        
#     print(frame.shape)    
#     frame = cv2.resize(frame, (int(frame.shape[1]*1.5), int(frame.shape[0]*1.5)))

#     frame = cv2.flip(frame, 1)
#     frame = cv2.flip(frame, 0)
    frame_r = frame
    
    fps = cam.get(cv2.CAP_PROP_FPS)
    cv2.putText(frame, f'fps_cv2: {fps}', (15, 15),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0),2, cv2.LINE_AA)
    
    
    # преобразуем изображение в чернобелый формат
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
   
    # детекция лиц
    faces = face_cascade.detectMultiScale(gray, 1.3, 6)
        
    # обновляем трекеры и получаем результат трекинга
    success_update_counter = 0
    
    if len(face_dict) != 0:
        
        for k, v  in list(face_dict.items()):
            tracker = v['tracker']
            success, box = tracker.update(frame)
                        
            if success:
                success_update_counter += 1
                v['tracker_box'] = box
                
            else:
                v['count_face_loss'] += 1
                v['count_traker_loss'] += 1
                v['tracker_box'] = None
                
            if v['count_face_loss'] > trashold_face_loss:
                # удаление трекера
                del face_dict[k]
                
    if len(faces) != 0 and len(face_dict) == 0:
        
        for i, face in enumerate(faces):
            
            if face is not None:
                (x, y, w, h) = map(int, face)
                box = None

                tracker = createTrackerByName(trackerType)
                tracker.init(frame, (x,y,w,h))

                face_dict.setdefault(face_id, {'face_box': face,
                                               'tracker': tracker,
                                               'tracker_box': box, 
                                               'count_face_loss': 0,
                                               'count_traker_loss': 0})
                face_id += 1


    elif len(faces) != 0 and len(face_dict) != 0:
        
        for i, face in enumerate(faces):
            (x, y, w, h) = map(int, face)

            matching_counter = 0
            for k, v  in list(face_dict.items()):
                                
                if v['tracker_box'] is not None:
                    tracker_box = v['tracker_box']
                    
                    iou = get_iou(face, tracker_box)
                    
                    if iou >= trashold_iou:
                        
                        matching_counter += 1
                        v['count_face_loss'] = 0
                        del v['tracker']
                        tracker = createTrackerByName(trackerType)
                        tracker.init(frame, (x,y,w,h))
                        v['tracker'] = tracker
                        v['face_box'] = face
                        v['count_traker_loss'] = 0
                        

            if matching_counter == 0: # новое лицо
                box = None
                tracker = createTrackerByName(trackerType)
                tracker.init(frame, (x,y,w,h))

                face_dict.setdefault(face_id, {'face_box': face,
                                               'tracker': tracker,
                                               'tracker_box': box, 
                                               'count_face_loss': 0,
                                               'count_traker_loss': 0,})
                face_id += 1

            # отрисовываем детекцию лиц
            cv2.rectangle(frame,(x,y),(x+w,y+h), (255,0,0),2)
            
        cv2.putText(frame, f'Detected: {len(faces)} face(s)', (15, 65),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0),2, cv2.LINE_AA)
    else:
        cv2.putText(frame, 'Faces not detected!', (15, 65),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0),2, cv2.LINE_AA)
 
    #control count line position
    px1 = frame.shape[1]-80
    py1 = 0
    px2 = frame.shape[1]-80
    py2 = frame.shape[0]
    cv2.line(frame, (px1,py1), (px2,py2), (0,0,255), 2)
    
    # отрисовываем результат трекера
    for k, v  in list(face_dict.items()):
        
        if v['tracker_box'] is not None:
            x, y, w, h = map(int, v['tracker_box'])
            
            # predict age and gender
            label_gen, label_age = pred_gender_age(x,y,w,h)
            
            cv2.rectangle(frame,(x,y),(x+w,y+h),color_dict[label_gen],2)
            cv2.rectangle(frame,(x,y-40),(x+w,y),color_dict[label_gen],-1)
            cv2.putText(frame, f'id={k}, {gender_dict[label_gen]} {label_age}', (x, y - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, 
                        (255, 255, 255), 2, cv2.LINE_AA)
            
            # people counter
            if x>px1:
                gender = gender_dict[label_gen]
                age = label_age
#                 face_img=frame_r[y:y+w,x:x+w]
                
                count_dict.setdefault(k, [gender, age])  # face_img, face_img.shape])
    
    num_frames += 1
    # End time
    end = time.time()
    # Time elapsed
    seconds = end - start
    # Calculate frames per second
    fps  = num_frames / seconds
    
    # fps
    cv2.putText(frame, f'fps: {round(fps, 1)}', (15, 45),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2, cv2.LINE_AA)
    cv2.putText(frame, f'Trackers: {len(face_dict)}', (15, 85),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2, cv2.LINE_AA)
    cv2.putText(frame, f'Count: {len(count_dict)}', (15, 105),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2, cv2.LINE_AA)
    
    # вывод инфо счетчика count_by_gender
    cv2.putText(frame, f'Male: {count_by_gender(count_dict)[0]}', (15, 125), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,255), 2, cv2.LINE_AA)
    cv2.putText(frame, f'Female: {count_by_gender(count_dict)[1]}', (15, 145), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2, cv2.LINE_AA)

    # show frame
    cv2.imshow('Face_Multi_Tracker', frame)
    
#     frame_width = int(cam.get(3))
#     frame_height = int(cam.get(4))
#     frame_size = (frame_width,frame_height)
    
#     output.write(frame) # запись видео
        
    # выход по нажатию на клавишу 'q'
    interrupt=cv2.waitKey(1)
    if  interrupt & 0xFF == ord('q'):
        break
        
cam.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 21ms/step


1/1 [==============================] - 0s 25ms/step


1/1 [==============================] - 0s 21ms/step


1/1 [==============================] - 0s 24ms/step


1/1 [==============================] - 0s 24ms/step


1/1 [==============================] - 0s 23ms/step


1/1 [==============================] - 0s 24ms/step


In [ ]:
cam.release()
cv2.destroyAllWindows()